In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import json

In [7]:
with open('../../../utilities/configurations/lyft_conection.json', 'r') as file:
    credenciales_lyft = json.load(file)

engine = create_engine(f"mysql+pymysql://{credenciales_lyft['USERNAME']}:{credenciales_lyft['PASSWORD']}@{credenciales_lyft['SERVER']}/{credenciales_lyft['DATABASE']}",
                       connect_args={
                            'init_command': "SET SESSION net_read_timeout=600, net_write_timeout=600, max_execution_time=3000000"
                        })

In [8]:
start_date = '2022-11-01 00:00:00'
end_date = '2025-11-30 23:59:59'

In [9]:
sql_BikeSubscriptionFact = f'''

SELECT
    -- 1. Construcción de la Fecha (Día a Día)
    d.year AS anio,
    d.month AS mes_numero,
    d.dayOfMonthValue AS dia_numero,      -- Columna específica para el día (1-31)
    d.dayOfWeek_localizedValue0 AS dia_semana, -- Ej: "Monday", "Sunday" (Útil para detectar estacionalidad semanal)

    -- 2. Segmentación (Tipo de Membresía)
    t.name_localizedValue1 AS tipo_suscripcion,
    t.id AS id_suscripcion,

    -- 3. Métricas Diarias
    COUNT(f.id) AS cantidad_Vendida,
    SUM(f.totalPaid) AS ingresos_totales_reales

FROM 
    BikeSubscriptionFact f

-- JOIN con DateDim para obtener el detalle del día
JOIN 
    DateDim d ON f.purchaseDate_id = d.id

-- JOIN para filtrar solo membresías
JOIN 
    BikeSubscriptionTypeDim t ON f.subscriptionType_id = t.id

WHERE 
    f.totalPaid > 0 
    AND d.year < 2026

GROUP BY 
    d.year, 
    d.month, 
    d.dayOfMonthValue,          -- Agrupación clave añadida
    d.dayOfWeek_localizedValue0,
    t.name_localizedValue1,
    t.id

ORDER BY 
    d.year ASC, 
    d.month ASC,
    d.dayOfMonthValue ASC;

'''

df_sql_BikeSubscriptionFact = pd.read_sql(sql_BikeSubscriptionFact, engine)

In [10]:
df_sql_BikeSubscriptionFact.head()

,anio,mes_numero,dia_numero,dia_semana,tipo_suscripcion,id_suscripcion,cantidad_Vendida,ingresos_totales_reales
0,2022,7,25,Monday,1 Día (I.V.A incluido),1,1,136.88
1,2022,7,27,Wednesday,3 Días (I.V.A incluido),2,1,234.00
2,2022,7,27,Wednesday,7 Días (I.V.A incluido),3,1,391.00
3,2022,7,28,Thursday,1 Día (I.V.A incluido),1,3,354.00
4,2022,7,28,Thursday,Anual (I.V.A incluido),4,1,521.00


In [11]:
df_sql_BikeSubscriptionFact['tipo_suscripcion'].unique()

array(['1 Día (I.V.A incluido)', '3 Días (I.V.A incluido)',
       '7 Días (I.V.A incluido)', 'Anual (I.V.A incluido)',
       'Membresía Plus TEST 01 (IVA incluido)',
       'Anual Ecobici+ (I.V.A incluido)', 'Ecobici HSBC', 'ECOBICI vivo'],
      dtype=object)

In [12]:
df_sql_BikeSubscriptionFact['fecha'] = df_sql_BikeSubscriptionFact['anio'].astype(str) + '-' + df_sql_BikeSubscriptionFact['mes_numero'].astype(str).str.zfill(2) + '-' + df_sql_BikeSubscriptionFact['dia_numero'].astype(str).str.zfill(2)
df_sql_BikeSubscriptionFact['fecha'] = pd.to_datetime(df_sql_BikeSubscriptionFact['fecha'], format='%Y-%m-%d')
df_sql_BikeSubscriptionFact = df_sql_BikeSubscriptionFact.sort_values(by='fecha')
df_sql_BikeSubscriptionFact.head()

,anio,mes_numero,dia_numero,dia_semana,tipo_suscripcion,id_suscripcion,cantidad_Vendida,ingresos_totales_reales,fecha
0,2022,7,25,Monday,1 Día (I.V.A incluido),1,1,136.88,2022-07-25
1,2022,7,27,Wednesday,3 Días (I.V.A incluido),2,1,234.00,2022-07-27
2,2022,7,27,Wednesday,7 Días (I.V.A incluido),3,1,391.00,2022-07-27
3,2022,7,28,Thursday,1 Día (I.V.A incluido),1,3,354.00,2022-07-28
4,2022,7,28,Thursday,Anual (I.V.A incluido),4,1,521.00,2022-07-28


In [14]:
df_sql_BikeSubscriptionFact.to_csv('../data/ingresos_membresia_diaria.csv', index=False)